### Libraries import

In [1]:
# Base libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
import functions as fun
import importlib as imp

# Libraries for plotting
import plotly.io as pio
import plotly.graph_objects as go

# Libraries for modelling
import statsmodels.api as sm
import sklearn.model_selection as modsel

# Set additional settings for warnings and templates
import warnings
warnings.filterwarnings("ignore")
pio.templates.default = "plotly_dark"
pd.set_option('display.max_columns', None)

In [26]:
# Reload functions if they were changed
imp.reload(fun)

<module 'functions' from 'c:\\Users\\User\\OneDrive\\PD\\ММСС\\early_warning_model\\functions.py'>

### Financial models

#### Feature generation

In [22]:
# Read dataset and define columns for feature generation
data = pd.read_parquet('Data/dataset.parquet').drop(columns = 'Variance')
indices = data.groupby(['Ticker', 'Index']).size().index.values
cols = ['Hurst', 'CorrDim', 'Lyapunov', 
        'Skewness', 'Kurtosis', 'PSD', 'ACF_1',
        'WL_C1', 'WL_C2', 'WL_C3']

# Set lag for dynamics and short variance calculation
lag_model = [8]

# Calculate dynamics and short variance
# Original idea about variance was born from the largest Lyapunov exponent's behaviour before the critical transition point:
# is mostly didn't move in nominal values but its variance in some cases decreased signigicantly 
data_logdyn = pd.DataFrame()
for ind in tqdm(indices):
    data_ind = data[(data['Ticker'] == ind[0]) & (data['Index'] == ind[1])]
    for col in cols:
        for lag_m in lag_model:
            data_ind[col + '_' + str(lag_m) + '_dyn'] = data_ind[col] / data_ind[col].shift(lag_m) - 1
            data_ind[col + '_' + str(lag_m) + '_Variance'] = data_ind[col].rolling(lag_m).var()
    data_ind.dropna(inplace = True)
    data_logdyn = pd.concat([data_logdyn, data_ind])

# Reset index to get rid of dates and save final dataset
data_logdyn.reset_index(drop = True, inplace = True)
data_logdyn = data_logdyn[data_logdyn['Distance'] > 0]
data_logdyn.to_parquet('Data/final_dataset.parquet')
data_logdyn

100%|██████████| 548/548 [00:17<00:00, 32.06it/s]


,Volume,MA100,MV100,Rise,Distance,Index,Ticker,Hurst,CorrDim,Lyapunov,Skewness,Kurtosis,PSD,ACF_1,WL_C1,WL_C2,WL_C3,Hurst_8_dyn,Hurst_8_Variance,CorrDim_8_dyn,CorrDim_8_Variance,Lyapunov_8_dyn,Lyapunov_8_Variance,Skewness_8_dyn,Skewness_8_Variance,Kurtosis_8_dyn,Kurtosis_8_Variance,PSD_8_dyn,PSD_8_Variance,ACF_1_8_dyn,ACF_1_8_Variance,WL_C1_8_dyn,WL_C1_8_Variance,WL_C2_8_dyn,WL_C2_8_Variance,WL_C3_8_dyn,WL_C3_8_Variance
0,23952.0,31070.20,7.130232e+08,False,291,2643,AAN,0.651904,-2.815337e-15,0.049050,4.481086,26.699359,-0.609296,0.374819,0.465914,-0.103608,-0.124637,-0.050382,0.002605,-0.391654,1.096794e-29,0.145294,0.000312,0.030011,0.002971,0.082711,0.754393,0.037030,0.000049,0.019152,0.000045,-0.045022,0.003071,0.285858,0.000373,-0.167933,0.001749
1,159410.0,32093.49,8.715063e+08,False,290,2643,AAN,0.662030,-9.789577e-17,0.048384,4.402063,25.787151,-0.612294,0.367293,0.418223,-0.130413,-0.086043,0.050584,0.002415,224.435763,1.094419e-29,0.111672,0.000318,0.012617,0.002558,0.046669,0.643472,0.038410,0.000053,-0.000925,0.000044,-0.009408,0.003095,0.559503,0.000450,-0.162758,0.001908
2,74965.0,32664.47,8.876980e+08,False,289,2643,AAN,0.690370,-2.362195e-15,0.017922,4.390049,25.683490,-0.614583,0.371478,0.475433,-0.103349,-0.175004,0.204455,0.000927,-1.947039,9.171144e-30,-0.676723,0.000350,0.011164,0.001981,0.044073,0.473688,0.038391,0.000054,0.009061,0.000048,-0.046778,0.002778,0.087733,0.000436,-0.030778,0.001826
3,57275.0,32998.33,8.929255e+08,False,288,2643,AAN,0.693400,-2.719365e-15,0.019476,4.389790,25.681349,-0.618153,0.370974,0.380598,-0.146165,-0.076268,0.038265,0.000894,-0.439684,7.705216e-30,-0.652200,0.000322,0.011460,0.001289,0.044543,0.257182,0.040634,0.000055,0.012792,0.000050,0.045300,0.002463,0.182044,0.000618,-0.023034,0.001851
4,36764.0,32491.61,8.628754e+08,False,287,2643,AAN,0.670705,2.596383e-15,0.050424,4.420547,25.969972,-0.621126,0.373620,0.466517,-0.100117,-0.133767,-0.013487,0.000923,-1.533414,6.951390e-30,-0.057433,0.000305,0.009323,0.001066,0.038771,0.127176,0.040629,0.000055,0.063141,0.000010,-0.045662,0.002215,0.182232,0.000544,-0.128684,0.001742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160010,103982.0,107127.65,4.066301e+09,False,5,1541,ZWS,0.762291,-3.139746e-15,0.002439,3.228418,15.896758,-0.752358,0.577192,0.573424,-0.154147,-0.074996,-0.010378,0.000124,-1.974779,-3.552214e-18,0.288016,0.000002,-0.003889,0.000036,-0.003274,0.001379,-0.021714,0.000026,-0.008208,0.000002,-0.019764,0.001104,0.024043,0.000198,0.047756,0.000524
160011,160370.0,108231.04,4.060776e+09,False,4,1541,ZWS,0.793100,-4.579490e-15,0.001399,3.229276,15.914075,-0.749824,0.576721,0.555407,-0.133274,-0.058564,0.032792,0.000163,-2.177920,-3.552214e-18,-0.411605,0.000003,-0.002980,0.000040,-0.002118,0.001451,-0.022367,0.000028,-0.008702,0.000003,-0.004787,0.001098,-0.038660,0.000224,0.090447,0.000535
160012,1031511.0,117603.93,1.258062e+10,False,3,1541,ZWS,0.767982,-4.548268e-15,0.006788,3.364231,16.524853,-0.747284,0.526691,0.563935,-0.163990,-0.059153,-0.028708,0.000136,-1.868521,-3.552214e-18,0.820707,0.000005,0.037579,0.002156,0.034430,0.045315,-0.023350,0.000031,-0.093385,0.000344,0.008998,0.001116,-0.023602,0.000208,0.327306,0.000546
160013,382418.0,120666.56,1.326214e+10,False,2,1541,ZWS,0.750742,2.858428e-15,0.006570,3.333177,16.262226,-0.744650,0.546938,0.506274,-0.149322,-0.015747,-0.014877,0.000191,-0.462384,-3.552214e-18,7.506477,0.000006,0.030473,0.002933,0.020689,0.052069,-0.024309,0.000034,-0.056829,0.000410,0.021562,0.000969,0.045151,0.000197,0.018425,0.000543


#### Probit modelling

##### Modelling with all variables

In the cell below we are iterating over the three lists of parameters:
- horizons - how many hours before the transition are considered to be close enough to be prediction phase
- sizes - share of the positive observations in the whole modelling dataset - this parameter is important because in the original dataset share of positives for some of the horizons was to small, so we decided to use decrease size of the negative dataset and randomize it
- states - in order to avoid lucky random choices in the sizes randomization we use a list of different random states to average the results

In [23]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
target = 'Flag'
horizons = list(range(1, 9))
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 500))

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states)

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_cols = list(coeffs_cols) + ['const']
res_means = fun.save_results(res, res_cols)
res_means

100%|██████████| 8/8 [17:04<00:00, 128.05s/it]


Train size  Test size  Train AUC  Test AUC  \
Horizon 1 Share 1 Share real                                               
1       0.05    0.050160          8740.0     2185.0    0.91705   0.90280   
        0.10    0.100293          4371.0     1093.0    0.91705   0.90460   
        0.15    0.150425          2914.0      729.0    0.91605   0.91330   
        0.20    0.200512          2186.0      547.0    0.91795   0.91220   
2       0.05    0.050328         17421.0     4356.0    0.85525   0.85080   
        0.10    0.100615          8714.0     2179.0    0.85725   0.85220   
        0.15    0.150881          5811.0     1453.0    0.85790   0.85020   
        0.20    0.201101          4360.0     1090.0    0.86165   0.84565   
3       0.05    0.050495         26046.0     6512.0    0.80690   0.80015   
        0.10    0.100933         13030.0     3258.0    0.80970   0.80575   
        0.15    0.151325          8691.0     2173.0    0.81185   0.80315   
        0.20    0.201644          6522.0     1631.0    0.81445   0.80665   
4       0.05    0.050661         34614.0     8654.0    0.76265   0.75410   
        0.10    0.101252         17319.0     4330.0    0.76555   0.76200   
        0.15    0.151769         11554.0     2889.0    0.76815   0.75880   
        0.20    0.202214          8672.0     2168.0    0.77100   0.75930   
5       0.05    0.050829         43124.0    10782.0    0.72515   0.72375   
        0.10    0.101568         21581.0     5396.0    0.73125   0.72225   
        0.15    0.152222         14400.0     3600.0    0.73515   0.72890   
        0.20    0.202783         10809.0     2703.0    0.73755   0.72835   
6       0.05    0.050999         51577.0    12895.0    0.70380   0.69525   
        0.10    0.101890         25816.0     6454.0    0.70760   0.70305   
        0.15    0.152675         17228.0     4308.0    0.71335   0.70270   
        0.20    0.203352         12935.0     3234.0    0.71445   0.70965   
7       0.05    0.051169         59973.0    14994.0    0.68530   0.68665   
        0.10    0.102212         30024.0     7506.0    0.69070   0.68685   
        0.15    0.153128         20040.0     5011.0    0.69515   0.69005   
        0.20    0.203923         15048.0     3763.0    0.69745   0.69415   
8       0.05    0.051340         68312.0    17079.0    0.66940   0.67870   
        0.10    0.102535         34204.0     8552.0    0.67710   0.67620   
        0.15    0.153587         22835.0     5709.0    0.68045   0.67915   
        0.20    0.204497         17150.0     4288.0    0.68260   0.67905   

                              Train KS-test p-value  Test KS-test p-value  \
Horizon 1 Share 1 Share real                                                
1       0.05    0.050160                        0.0                   0.0   
        0.10    0.100293                        0.0                   0.0   
        0.15    0.150425                        0.0                   0.0   
        0.20    0.200512                        0.0                   0.0   
2       0.05    0.050328                        0.0                   0.0   
        0.10    0.100615                        0.0                   0.0   
        0.15    0.150881                        0.0                   0.0   
        0.20    0.201101                        0.0                   0.0   
3       0.05    0.050495                        0.0                   0.0   
        0.10    0.100933                        0.0                   0.0   
        0.15    0.151325                        0.0                   0.0   
        0.20    0.201644                        0.0                   0.0   
4       0.05    0.050661                        0.0                   0.0   
        0.10    0.101252                        0.0                   0.0   
        0.15    0.151769                        0.0                   0.0   
        0.20    0.202214                        0.0                   0.0   
5       0.05    0.050829                        0.0        

In [24]:
# Get mean metrics for all of the columns to understand what variables are actually used in the final models
pd.DataFrame(round(res_means[np.in1d(res_means.index.get_level_values(0), list(range(1, 5)))].mean(), 2), columns = ['Average coeff'])
# round(res_means.mean(), 2)

,Average coeff
Train size,26860.75
Test size,6715.75
Train AUC,0.72
Test AUC,0.71
Train KS-test p-value,0.00
Test KS-test p-value,0.00
Train F1-score,0.42
Test F1-score,0.42
Train precision,0.42
Test precision,0.42


In [17]:
# Count a share of appearances for positive horizons
res = pd.read_parquet('Params/params_Probit.parquet')
res_1_5 = res[res['Horizon'].isin(range(1, 5))]
groups = ['Horizon', '1 Share', '1 Share real']
drops = ['State']
round(res_1_5.groupby(groups)[list(coeffs_cols) + ['const']].count() / 20, 2)
# round(res.groupby(groups)[list(coeffs_cols) + ['const']].count() / 20, 2)

Hurst  CorrDim  Lyapunov  Skewness  Kurtosis  \
Horizon 1 Share 1 Share real                                                 
1       0.05    0.050160       0.00     0.05      1.00       1.0       1.0   
        0.10    0.100293       0.00     0.15      0.90       1.0       1.0   
        0.15    0.150425       0.00     0.05      0.75       1.0       1.0   
        0.20    0.200512       0.10     0.15      0.65       1.0       1.0   
2       0.05    0.050328       0.00     0.00      1.00       1.0       1.0   
        0.10    0.100615       0.00     0.15      0.90       1.0       1.0   
        0.15    0.150881       0.00     0.05      0.85       1.0       1.0   
        0.20    0.201101       0.00     0.10      0.85       1.0       1.0   
3       0.05    0.050495       0.00     0.20      0.95       1.0       1.0   
        0.10    0.100933       0.00     0.25      0.95       1.0       1.0   
        0.15    0.151325       0.00     0.30      0.95       1.0       1.0   
        0.20    0.201644       0.05     0.10      0.70       1.0       1.0   
4       0.05    0.050661       0.00     0.05      1.00       1.0       1.0   
        0.10    0.101252       0.00     0.20      1.00       1.0       1.0   
        0.15    0.151769       0.00     0.05      0.95       1.0       1.0   
        0.20    0.202214       0.00     0.20      0.80       1.0       1.0   

                               PSD  ACF_1  WL_C1  WL_C2  WL_C3  Hurst_8_dyn  \
Horizon 1 Share 1 Share real                                                  
1       0.05    0.050160      0.80   0.70   0.10   0.25    1.0         0.15   
        0.10    0.100293      0.75   0.45   0.15   0.15    1.0         0.00   
        0.15    0.150425      0.65   0.25   0.20   0.20    1.0         0.00   
        0.20    0.200512      0.75   0.15   0.10   0.00    1.0         0.00   
2       0.05    0.050328      1.00   1.00   0.80   0.60    1.0         0.00   
        0.10    0.100615      1.00   0.90   0.80   0.45    1.0         0.00   
        0.15    0.150881      0.95   0.75   0.65   0.45    1.0         0.00   
        0.20    0.201101      0.95   0.65   0.65   0.40    1.0         0.05   
3       0.05    0.050495      1.00   0.90   0.60   0.60    1.0         0.00   
        0.10    0.100933      1.00   0.85   0.65   0.50    1.0         0.00   
        0.15    0.151325      1.00   0.75   0.65   0.60    1.0         0.00   
        0.20    0.201644      1.00   0.70   0.85   0.55    1.0         0.00   
4       0.05    0.050661      1.00   1.00   0.65   1.00    1.0         0.00   
        0.10    0.101252      1.00   0.85   0.60   0.95    1.0         0.00   
        0.15    0.151769      1.00   0.85   0.80   0.80    1.0         0.10   
        0.20    0.202214      1.00   0.60   0.65   0.75    1.0         0.10   

                              Hurst_8_Variance  CorrDim_8_dyn  \
Horizon 1 Share 1 Share real                                    
1       0.05    0.050160                  0.50           0.10   
        0.10    0.100293                  0.20           0.05   
        0.15    0.150425                  0.30           0.15   
        0.20    0.200512                  0.20           0.20   
2       0.05    0.050328                  0.75           0.00   
        0.10    0.100615                  0.80           0.15   
        0.15    0.150881                  0.60           0.30   
        0.20    0.201101                  0.55           0.15   
3       0.05    0.050495                  1.00           0.00   
        0.10    0.100933                  1.00           0.00   
        0.15    0.151325                  0.90           0.05   
        0.20    0.201644                  0.95           0.05   
4       0.05    0.050661                  1.00           0.00   
        0.10    0.101252                  0.95           0.00   
        0.15    0.151769                  1.00           0.05   
        0.20    0.202214                  0.95           0.00   

                              C

In [18]:
# Count an aggregated share of appearances
pd.DataFrame(round(res_1_5[list(coeffs_cols) + ['const']].count() / len(res_1_5), 2), columns = ['Share'])
# pd.DataFrame(round(res[list(coeffs_cols) + ['const']].count() / len(res), 2), columns = ['Share'])

,Share
Hurst,0.01
CorrDim,0.13
Lyapunov,0.89
Skewness,1.00
Kurtosis,1.00
PSD,0.93
ACF_1,0.71
WL_C1,0.56
WL_C2,0.52
WL_C3,1.00


##### Visualization for the KS-test

In [27]:
# Vizual check of the single model
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')
target = 'Flag'
horizon = 8
share = 0.1
state = 2000
data_testing = data_logdyn.copy()
data_testing['Flag'] = data_testing['Distance'].apply(lambda x: 0 if x >= horizon else 1)
data_testing.drop(columns = ['Volume', 'MA100', 'MA100','Rise', 'Distance', 'Index', 'Ticker'], inplace = True)

data_testing_1 = data_testing[data_testing[target] == 1]
data_testing_0 = data_testing[data_testing[target] == 0]
Y_1 = data_testing_1[target]
X_1 = data_testing_1.drop(columns = [target])
share_1_orig = len(data_testing_1) / (len(data_testing_0) + len(data_testing_1))
_, X_0, _, Y_0 = modsel.train_test_split(data_testing_0.drop(columns = [target]), data_testing_0[target], 
                                                     test_size = min(share_1_orig * (1 - share) / share, 1), random_state = state)
share_1 = len(Y_1) / (len(Y_0) + len(Y_1))
Y = pd.concat([Y_0, Y_1])
X = sm.add_constant(pd.concat([X_0, X_1]))
X_train, X_test, Y_train, Y_test = modsel.train_test_split(X, Y, test_size = 0.2, random_state = state)
results_rs, auc_train_rs, auc_test_rs, ks_train_rs, ks_test_rs, f1_train_rs,\
    f1_test_rs, pr_train_rs, pr_test_rs, rec_train_rs, rec_test_rs\
    = fun.model_optimization(Y_train, Y_test, X_train, X_test, silent = True)
print(results_rs.summary())
Y_test_pred = results_rs.predict(X_test)
ks_samples = pd.DataFrame({'Y': Y_test, 'Y_pred': Y_test_pred})
ks_samples_posi = ks_samples[ks_samples['Y'] == 1]['Y_pred']
ks_samples_nega = ks_samples[ks_samples['Y'] == 0]['Y_pred']
fig = go.Figure()
fig.add_trace(go.Histogram(x = ks_samples_posi, name = 'Posi', nbinsx = 30))
fig.add_trace(go.Histogram(x = ks_samples_nega, name = 'Nega', nbinsx = 100))
fig.update_layout(barmode = 'overlay')
fig.update_traces(opacity = 0.75)
fig.show()

                          Probit Regression Results                           
Dep. Variable:                   Flag   No. Observations:                30024
Model:                         Probit   Df Residuals:                    30004
Method:                           MLE   Df Model:                           19
Date:                Wed, 07 Aug 2024   Pseudo R-squ.:                  0.1584
Time:                        14:16:27   Log-Likelihood:                -8320.4
converged:                       True   LL-Null:                       -9886.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -1.7121      0.074    -23.170      0.000      -1.857      -1.567
Lyapunov                3.0276      0.860      3.520      0.000       1.342       4.714
Skewness        

##### Modelling with separate variables

In [3]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
target = 'Flag'
horizons = list(range(1, 6))
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 500))

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res_sep = fun.model(data_logdyn, target, horizons, shares, states, separate = True)

# Save results and create pivot based on the horizon and 1 share parameters
res_means = fun.save_results(res, [], sep = True)
res_means[res_means['Test AUC'] >= 0.75].sort_values('Test AUC', ascending = False)

100%|██████████| 5/5 [16:58<00:00, 203.76s/it]


,,Train size,Test size,Train AUC,Test AUC,Train KS-test p-value,Test KS-test p-value,Train F1-score,Test F1-score,Train precision,Test precision,Train recall,Test recall,Const,Const_Pvalue,Coef,Coef_Pvalue
Variable,Horizon,,,,,,,,,,,,,,,,
Skewness_8_Variance,1,4648.565789,1162.447368,0.940487,0.941184,0.0,0.0,0.703316,0.711487,0.612789,0.625868,0.850697,0.862592,-1.512770,3.296619e-217,0.652842,5.436307e-46
Kurtosis_8_Variance,1,4709.863014,1177.767123,0.936753,0.937918,0.0,0.0,0.666247,0.679959,0.561397,0.583945,0.850932,0.859603,-1.466881,2.058904e-204,0.001631,2.383182e-43
ACF_1_8_Variance,1,4552.750000,1138.500000,0.930875,0.931375,0.0,0.0,0.754200,0.738487,0.706313,0.673013,0.825500,0.845675,-1.660177,1.426042e-232,793.145150,4.934820e-73
Skewness_8_Variance,2,9501.500000,2375.771429,0.895057,0.896329,0.0,0.0,0.635514,0.614543,0.568829,0.528614,0.749100,0.773514,-1.462848,0.000000e+00,0.813546,1.348995e-84
Kurtosis_8_Variance,2,9918.949153,2480.152542,0.890254,0.890881,0.0,0.0,0.577915,0.598424,0.486559,0.514203,0.759136,0.758661,-1.438510,0.000000e+00,0.001947,5.874377e-74
ACF_1_8_Variance,2,9076.500000,2269.500000,0.882225,0.882237,0.0,0.0,0.649175,0.639925,0.600300,0.584213,0.730000,0.747825,-1.558918,0.000000e+00,860.292515,4.765923e-118
Skewness_8_Variance,3,14732.015385,3683.446154,0.860585,0.857877,0.0,0.0,0.526231,0.545954,0.439615,0.474785,0.704538,0.697769,-1.429076,0.000000e+00,0.765947,1.926762e-104
Kurtosis_8_Variance,3,16676.297872,4169.553191,0.853809,0.850979,0.0,0.0,0.479596,0.488000,0.382043,0.393234,0.701830,0.703000,-1.460052,0.000000e+00,0.001748,3.609335e-90
ACF_1_8_Variance,3,13572.250000,3393.500000,0.842025,0.839112,0.0,0.0,0.554700,0.565813,0.494675,0.504663,0.676063,0.683462,-1.477161,0.000000e+00,802.732695,1.266908e-153


In [12]:
# Getting highest AUC values for variables
res_sep = pd.read_parquet('Data/params_sep.parquet')
res_sep.groupby(groups)['Test AUC'].mean().to_frame().reset_index(1, drop = False).groupby('Variable')['Test AUC'].max().sort_values(ascending = False).round(2)

Variable
Skewness_8_Variance    0.94
Kurtosis_8_Variance    0.94
ACF_1_8_Variance       0.93
Skewness               0.81
Skewness_8_dyn         0.81
Kurtosis               0.80
Kurtosis_8_dyn         0.77
Lyapunov_8_Variance    0.76
Lyapunov               0.74
CorrDim                0.64
CorrDim_8_Variance     0.58
PSD_8_dyn              0.57
WL_C1_8_dyn            0.57
ACF_1                  0.55
WL_C1                  0.54
PSD_8_Variance         0.54
WL_C2                  0.53
WL_C2_8_Variance       0.53
PSD                    0.52
ACF_1_8_dyn            0.52
Hurst_8_Variance       0.51
WL_C1_8_Variance       0.51
WL_C3                  0.51
Hurst_8_dyn            0.51
WL_C3_8_dyn            0.51
WL_C2_8_dyn            0.51
Hurst                  0.50
WL_C3_8_Variance       0.50
CorrDim_8_dyn          0.50
Lyapunov_8_dyn         0.49
Name: Test AUC, dtype: float64

#### Random forest modelling

In [2]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
model = 'RF'
target = 'Flag'
horizons = list(range(1, 13))
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 50))

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model)

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model)
res_means

100%|██████████| 12/12 [16:24:13<00:00, 4921.12s/it]  


Train size  Test size  Train AUC  Test AUC  \
Horizon 1 Share 1 Share real                                               
1       0.05    0.050160          8740.0     2185.0   0.971230  0.950240   
        0.10    0.100293          4371.0     1093.0   0.974360  0.950715   
        0.15    0.150425          2914.0      729.0   0.976155  0.952500   
        0.20    0.200512          2186.0      547.0   0.978210  0.951085   
2       0.05    0.050328         17421.0     4356.0   0.934575  0.915465   
        0.10    0.100615          8714.0     2179.0   0.938615  0.916590   
        0.15    0.150881          5811.0     1453.0   0.940520  0.919115   
        0.20    0.201101          4360.0     1090.0   0.942990  0.917410   
3       0.05    0.050495         26046.0     6512.0   0.901630  0.883500   
        0.10    0.100933         13030.0     3258.0   0.905925  0.886650   
        0.15    0.151325          8691.0     2173.0   0.908465  0.887180   
        0.20    0.201644          6522.0     1631.0   0.910860  0.888405   
4       0.05    0.050661         34614.0     8654.0   0.868295  0.853455   
        0.10    0.101252         17319.0     4330.0   0.873475  0.855545   
        0.15    0.151769         11554.0     2889.0   0.875985  0.857110   
        0.20    0.202214          8672.0     2168.0   0.878440  0.857530   
5       0.05    0.050829         43124.0    10782.0   0.833515  0.819760   
        0.10    0.101568         21581.0     5396.0   0.838835  0.821460   
        0.15    0.152222         14400.0     3600.0   0.842150  0.823230   
        0.20    0.202783         10809.0     2703.0   0.845050  0.825630   
6       0.05    0.050999         51577.0    12895.0   0.810195  0.797100   
        0.10    0.101890         25816.0     6454.0   0.814965  0.799840   
        0.15    0.152675         17228.0     4308.0   0.818730  0.799840   
        0.20    0.203352         12935.0     3234.0   0.821465  0.802720   
7       0.05    0.051169         59973.0    14994.0   0.791515  0.777315   
        0.10    0.102212         30024.0     7506.0   0.796195  0.780785   
        0.15    0.153128         20040.0     5011.0   0.799880  0.782645   
        0.20    0.203923         15048.0     3763.0   0.802575  0.784340   
8       0.05    0.051340         68312.0    17079.0   0.777240  0.764935   
        0.10    0.102535         34204.0     8552.0   0.781630  0.767520   
        0.15    0.153587         22835.0     5709.0   0.784965  0.770600   
        0.20    0.204497         17150.0     4288.0   0.788320  0.770770   
9       0.05    0.051513         76593.0    19149.0   0.764780  0.750915   
        0.10    0.102861         38358.0     9590.0   0.769310  0.753430   
        0.15    0.154048         25612.0     6404.0   0.772290  0.756940   
        0.20    0.205073         19240.0     4810.0   0.774855  0.759535   
10      0.05    0.051687         84817.0    21205.0   0.752665  0.740220   
        0.10    0.103190         42484.0    10622.0   0.757030  0.744180   
        0.15    0.154510         28373.0     7094.0   0.760735  0.744900   
        0.20    0.205652         21317.0     5330.0   0.763280  0.746795   
11      0.05    0.051862         92984.0    23247.0   0.742030  0.730820   
        0.10    0.103521         46584.0    11646.0   0.746275  0.733990   
        0.15    0.154977         31116.0     7780.0   0.749355  0.736085   
        0.20    0.206234         23383.0     5846.0   0.752270  0.736105   
12      0.05    0.052038        101094.0    25274.0   0.732975  0.720610   
        0.10    0.103853         50656.0    12664.0   0.737085  0.724185   
        0.15    0.155446         33843.0     8461.0   0.740105  0.727240   
        0.20    0.206818         25436.0     6360.0   0.742945  0.728100   

                              Train KS-test p-value  Test KS-test p-value  \
Horizon 1 Share 1 Share real                                                
1       0.05    0.050160                        0.0                   0.0  

In [19]:
# Count an average feature importance
res = pd.read_parquet(f'Params/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 10))]
groups = ['Horizon']

# Get the final pivot and multiply by 100000 to make results more readable
means = round(res_viable[coeffs_cols].mean() * 100000, 2)
print(round(means.mean(), 2), round(means.median(), 2))
means

36.79 6.88


Hurst                    0.75
CorrDim                  2.28
Lyapunov                28.72
Skewness                58.21
Kurtosis                28.64
PSD                      7.98
ACF_1                    7.13
WL_C1                    6.62
WL_C2                    3.23
WL_C3                    4.36
Hurst_8_dyn              0.65
Hurst_8_Variance         2.38
CorrDim_8_dyn            4.72
CorrDim_8_Variance       0.85
Lyapunov_8_dyn           4.97
Lyapunov_8_Variance     18.06
Skewness_8_dyn         147.36
Skewness_8_Variance    132.64
Kurtosis_8_dyn          85.83
Kurtosis_8_Variance    123.03
PSD_8_dyn               14.87
PSD_8_Variance           4.12
ACF_1_8_dyn            116.87
ACF_1_8_Variance       256.66
WL_C1_8_dyn             12.80
WL_C1_8_Variance         3.30
WL_C2_8_dyn              1.23
WL_C2_8_Variance         5.60
WL_C3_8_dyn              2.61
WL_C3_8_Variance        17.19
dtype: float64

#### SVM modelling

In [4]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
model = 'SVM'
target = 'Flag'
horizons = [1]
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 50))

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model)

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model)
res_means

100%|██████████| 1/1 [1:45:53<00:00, 6353.89s/it]


Train size  Test size  Train AUC  Test AUC  \
Horizon 1 Share 1 Share real                                               
1       0.05    0.050160          8740.0     2185.0    0.56989  0.565310   
        0.10    0.100293          4371.0     1093.0    0.56180  0.560750   
        0.15    0.150425          2914.0      729.0    0.55749  0.553055   
        0.20    0.200512          2186.0      547.0    0.55734  0.551890   

                              Train KS-test p-value  Test KS-test p-value  \
Horizon 1 Share 1 Share real                                                
1       0.05    0.050160               2.276500e-08              0.013104   
        0.10    0.100293               1.123800e-07              0.014266   
        0.15    0.150425               3.480400e-07              0.030260   
        0.20    0.200512               9.266850e-07              0.028407   

                              Train F1-score  Test F1-score  Train precision  \
Horizon 1 Share 1 Share real                                                   
1       0.05    0.050160            0.222320       0.221125         0.347470   
        0.10    0.100293            0.271980       0.278935         0.477710   
        0.15    0.150425            0.293475       0.300085         0.531065   
        0.20    0.200512            0.314880       0.322255         0.564635   

                              Test precision  Train recall  Test recall  \
Horizon 1 Share 1 Share real                                              
1       0.05    0.050160            0.341625      0.250755     0.274720   
        0.10    0.100293            0.476365      0.242390     0.264665   
        0.15    0.150425            0.541495      0.249060     0.266310   
        0.20    0.200512            0.586075      0.260120     0.272350   

                              Hurst  CorrDim  Lyapunov  Skewness  Kurtosis  \
Horizon 1 Share 1 Share real                                                 
1       0.05    0.050160        0.0      0.0       0.0       0.0       0.0   
        0.10    0.100293        0.0      0.0       0.0       0.0       0.0   
        0.15    0.150425        0.0      0.0       0.0       0.0       0.0   
        0.20    0.200512        0.0      0.0       0.0       0.0       0.0   

                              PSD  ACF_1  WL_C1  WL_C2  WL_C3  Hurst_8_dyn  \
Horizon 1 Share 1 Share real                                                 
1       0.05    0.050160      0.0    0.0    0.0    0.0    0.0          0.0   
        0.10    0.100293      0.0    0.0    0.0    0.0    0.0          0.0   
        0.15    0.150425      0.0    0.0    0.0    0.0    0.0          0.0   
        0.20    0.200512      0.0    0.0    0.0    0.0    0.0          0.0   

                              Hurst_8_Variance  CorrDim_8_dyn  \
Horizon 1 Share 1 Share real                                    
1       0.05    0.050160                   0.0       0.005638   
        0.10    0.100293                   0.0       0.010282   
        0.15    0.150425                   0.0       0.012715   
        0.20    0.200512                   0.0       0.014759   

                              CorrDim_8_Variance  Lyapunov_8_dyn  \
Horizon 1 Share 1 Share real                                       
1       0.05    0.050160                     0.0             0.0   
        0.10    0.100293                     0.0             0.0   
        0.15    0.150425                     0.0             0.0   
        0.20    0.200512                     0.0             0.0   

                              Lyapunov_8_Variance  Skewness_8_dyn  \
Horizon 1 Share 1 Share real                                        
1       0.05    0.050160                      0.0             0.0   
        0.10    0.100293                      0.0             0.0   
        0.15    0.150425                      0.0             0.0   
        0.20    0.200512                      0.0             0.0   

                         

In [21]:
# Count an average feature importance
res = pd.read_parquet(f'Params/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 2))]
groups = ['Horizon']

# Get the final pivot and multiply by 100000 to make results more readable
means = round(res_viable[coeffs_cols].mean() * 100000, 2)
print(round(means.mean(), 2), round(means.median(), 2))
means

36.16 0.0


Hurst                     0.00
CorrDim                   0.00
Lyapunov                  0.00
Skewness                  0.00
Kurtosis                  0.00
PSD                       0.00
ACF_1                     0.00
WL_C1                     0.00
WL_C2                     0.00
WL_C3                     0.00
Hurst_8_dyn               0.00
Hurst_8_Variance          0.00
CorrDim_8_dyn          1084.83
CorrDim_8_Variance        0.00
Lyapunov_8_dyn            0.00
Lyapunov_8_Variance       0.00
Skewness_8_dyn            0.00
Skewness_8_Variance       0.00
Kurtosis_8_dyn            0.00
Kurtosis_8_Variance       0.00
PSD_8_dyn                 0.00
PSD_8_Variance            0.00
ACF_1_8_dyn               0.00
ACF_1_8_Variance          0.00
WL_C1_8_dyn               0.00
WL_C1_8_Variance          0.00
WL_C2_8_dyn               0.00
WL_C2_8_Variance          0.00
WL_C3_8_dyn               0.00
WL_C3_8_Variance          0.00
dtype: float64

#### Gradient boosting modelling

##### LightGBM

In [13]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
model = 'LightGBM'
target = 'Flag'
horizons = list(range(1, 16))
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 50))

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model)

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model)
res_means

100%|██████████| 15/15 [1:24:18<00:00, 337.22s/it]


Train size  Test size  Train AUC  Test AUC  \
Horizon 1 Share 1 Share real                                               
1       0.05    0.050160          8740.0     2185.0   0.965030  0.957085   
        0.10    0.100293          4371.0     1093.0   0.963585  0.956535   
        0.15    0.150425          2914.0      729.0   0.964905  0.957765   
        0.20    0.200512          2186.0      547.0   0.965420  0.955530   
2       0.05    0.050328         17421.0     4356.0   0.939720  0.922665   
        0.10    0.100615          8714.0     2179.0   0.941555  0.921560   
        0.15    0.150881          5811.0     1453.0   0.942005  0.923280   
        0.20    0.201101          4360.0     1090.0   0.943510  0.921390   
3       0.05    0.050495         26046.0     6512.0   0.916855  0.893575   
        0.10    0.100933         13030.0     3258.0   0.917745  0.895170   
        0.15    0.151325          8691.0     2173.0   0.918850  0.894780   
        0.20    0.201644          6522.0     1631.0   0.919715  0.895165   
4       0.05    0.050661         34614.0     8654.0   0.886000  0.866520   
        0.10    0.101252         17319.0     4330.0   0.887735  0.866360   
        0.15    0.151769         11554.0     2889.0   0.889190  0.868080   
        0.20    0.202214          8672.0     2168.0   0.890065  0.866465   
5       0.05    0.050829         43124.0    10782.0   0.856920  0.838770   
        0.10    0.101568         21581.0     5396.0   0.858350  0.838195   
        0.15    0.152222         14400.0     3600.0   0.859430  0.838320   
        0.20    0.202783         10809.0     2703.0   0.860300  0.839190   
6       0.05    0.050999         51577.0    12895.0   0.835650  0.818770   
        0.10    0.101890         25816.0     6454.0   0.836760  0.818270   
        0.15    0.152675         17228.0     4308.0   0.838270  0.817585   
        0.20    0.203352         12935.0     3234.0   0.839265  0.818345   
7       0.05    0.051169         59973.0    14994.0   0.817820  0.800195   
        0.10    0.102212         30024.0     7506.0   0.818960  0.800970   
        0.15    0.153128         20040.0     5011.0   0.820380  0.800885   
        0.20    0.203923         15048.0     3763.0   0.820860  0.801740   
8       0.05    0.051340         68312.0    17079.0   0.804650  0.788825   
        0.10    0.102535         34204.0     8552.0   0.805470  0.788330   
        0.15    0.153587         22835.0     5709.0   0.806300  0.789960   
        0.20    0.204497         17150.0     4288.0   0.807315  0.788645   
9       0.05    0.051513         76593.0    19149.0   0.793400  0.776755   
        0.10    0.102861         38358.0     9590.0   0.794140  0.776375   
        0.15    0.154048         25612.0     6404.0   0.794400  0.777715   
        0.20    0.205073         19240.0     4810.0   0.794620  0.777905   
10      0.05    0.051687         84817.0    21205.0   0.783600  0.768245   
        0.10    0.103190         42484.0    10622.0   0.783635  0.768505   
        0.15    0.154510         28373.0     7094.0   0.784210  0.766985   
        0.20    0.205652         21317.0     5330.0   0.784085  0.767310   
11      0.05    0.051862         92984.0    23247.0   0.774055  0.759765   
        0.10    0.103521         46584.0    11646.0   0.773900  0.759400   
        0.15    0.154977         31116.0     7780.0   0.774185  0.759965   
        0.20    0.206234         23383.0     5846.0   0.774760  0.757830   
12      0.05    0.052038        101094.0    25274.0   0.766110  0.750635   
        0.10    0.103853         50656.0    12664.0   0.765650  0.751290   
        0.15    0.155446         33843.0     8461.0   0.765850  0.751605   
        0.20    0.206818         25436.0     6360.0   0.766470  0.750865   
13      0.05    0.052216        109147.0    27287.0   0.757855  0.744815   
        0.10    0.104189         54700.0    13676.0   0.757660  0.742910   
        0.15    0.155920         36552.0     9138.0   0.757740  0.742335   
  

In [23]:
# Count an average feature importance
res = pd.read_parquet(f'Params/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 14))]
groups = ['Horizon']

# Get the final pivot
means = round(res_viable[coeffs_cols].mean(), 2)
print(round(means.mean(), 2), round(means.median(), 2))
means

794.44 167.46


Hurst                    22.86
CorrDim                  46.15
Lyapunov                224.01
Skewness                782.72
Kurtosis                149.56
PSD                     382.55
ACF_1                   185.35
WL_C1                    47.60
WL_C2                   188.48
WL_C3                    64.75
Hurst_8_dyn               7.09
Hurst_8_Variance         55.64
CorrDim_8_dyn            16.61
CorrDim_8_Variance      469.18
Lyapunov_8_dyn           11.48
Lyapunov_8_Variance     136.60
Skewness_8_dyn         6881.82
Skewness_8_Variance    4016.62
Kurtosis_8_dyn         1497.51
Kurtosis_8_Variance     547.93
PSD_8_dyn              1152.50
PSD_8_Variance          147.14
ACF_1_8_dyn             404.47
ACF_1_8_Variance       5466.74
WL_C1_8_dyn             412.06
WL_C1_8_Variance         89.66
WL_C2_8_dyn              24.32
WL_C2_8_Variance        119.24
WL_C3_8_dyn              16.34
WL_C3_8_Variance        266.21
dtype: float64

##### XGBoost

In [24]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
model = 'XGBoost'
target = 'Flag'
horizons = list(range(1, 16))
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 50))

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model)

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model)
res_means

100%|██████████| 15/15 [1:52:28<00:00, 449.92s/it]


Train size  Test size  Train AUC  Test AUC  \
Horizon 1 Share 1 Share real                                               
1       0.05    0.050160          8740.0     2185.0   0.965655  0.957605   
        0.10    0.100293          4371.0     1093.0   0.966400  0.956785   
        0.15    0.150425          2914.0      729.0   0.966845  0.958055   
        0.20    0.200512          2186.0      547.0   0.966410  0.955255   
2       0.05    0.050328         17421.0     4356.0   0.954865  0.925770   
        0.10    0.100615          8714.0     2179.0   0.956005  0.924300   
        0.15    0.150881          5811.0     1453.0   0.956920  0.925690   
        0.20    0.201101          4360.0     1090.0   0.959145  0.923405   
3       0.05    0.050495         26046.0     6512.0   0.941325  0.902150   
        0.10    0.100933         13030.0     3258.0   0.943390  0.902135   
        0.15    0.151325          8691.0     2173.0   0.946390  0.902260   
        0.20    0.201644          6522.0     1631.0   0.947275  0.901995   
4       0.05    0.050661         34614.0     8654.0   0.916975  0.880135   
        0.10    0.101252         17319.0     4330.0   0.920500  0.879530   
        0.15    0.151769         11554.0     2889.0   0.923530  0.880825   
        0.20    0.202214          8672.0     2168.0   0.925485  0.879500   
5       0.05    0.050829         43124.0    10782.0   0.892445  0.857010   
        0.10    0.101568         21581.0     5396.0   0.896310  0.856715   
        0.15    0.152222         14400.0     3600.0   0.899235  0.857245   
        0.20    0.202783         10809.0     2703.0   0.901330  0.857310   
6       0.05    0.050999         51577.0    12895.0   0.875415  0.841705   
        0.10    0.101890         25816.0     6454.0   0.878260  0.842185   
        0.15    0.152675         17228.0     4308.0   0.881605  0.841995   
        0.20    0.203352         12935.0     3234.0   0.883545  0.841325   
7       0.05    0.051169         59973.0    14994.0   0.858855  0.826240   
        0.10    0.102212         30024.0     7506.0   0.862515  0.827440   
        0.15    0.153128         20040.0     5011.0   0.865420  0.827875   
        0.20    0.203923         15048.0     3763.0   0.867445  0.828420   
8       0.05    0.051340         68312.0    17079.0   0.846520  0.816820   
        0.10    0.102535         34204.0     8552.0   0.849815  0.817265   
        0.15    0.153587         22835.0     5709.0   0.852245  0.819110   
        0.20    0.204497         17150.0     4288.0   0.854650  0.817390   
9       0.05    0.051513         76593.0    19149.0   0.835090  0.806510   
        0.10    0.102861         38358.0     9590.0   0.838355  0.806980   
        0.15    0.154048         25612.0     6404.0   0.840525  0.808555   
        0.20    0.205073         19240.0     4810.0   0.842425  0.808925   
10      0.05    0.051687         84817.0    21205.0   0.825925  0.799420   
        0.10    0.103190         42484.0    10622.0   0.828830  0.800935   
        0.15    0.154510         28373.0     7094.0   0.831175  0.800350   
        0.20    0.205652         21317.0     5330.0   0.832765  0.800525   
11      0.05    0.051862         92984.0    23247.0   0.817165  0.792530   
        0.10    0.103521         46584.0    11646.0   0.820305  0.793980   
        0.15    0.154977         31116.0     7780.0   0.822190  0.794820   
        0.20    0.206234         23383.0     5846.0   0.824355  0.792605   
12      0.05    0.052038        101094.0    25274.0   0.810380  0.785180   
        0.10    0.103853         50656.0    12664.0   0.813015  0.787350   
        0.15    0.155446         33843.0     8461.0   0.814735  0.787800   
        0.20    0.206818         25436.0     6360.0   0.816840  0.787460   
13      0.05    0.052216        109147.0    27287.0   0.802635  0.779845   
        0.10    0.104189         54700.0    13676.0   0.805640  0.780220   
        0.15    0.155920         36552.0     9138.0   0.807650  0.780490   
  

In [25]:
# Count an average feature importance
res = pd.read_parquet(f'Params/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 16))]
groups = ['Horizon']

# Get the final pivot
means = round(res_viable[coeffs_cols].mean(), 2)
print(round(means.mean(), 2), round(means.median(), 2))
means

33.41 16.29


Hurst                   12.82
CorrDim                 10.80
Lyapunov                18.29
Skewness                25.77
Kurtosis                17.52
PSD                     16.46
ACF_1                   17.85
WL_C1                   12.15
WL_C2                   12.48
WL_C3                   11.69
Hurst_8_dyn              7.25
Hurst_8_Variance        14.35
CorrDim_8_dyn            8.56
CorrDim_8_Variance      20.57
Lyapunov_8_dyn           7.99
Lyapunov_8_Variance     14.26
Skewness_8_dyn         184.81
Skewness_8_Variance    112.35
Kurtosis_8_dyn          43.51
Kurtosis_8_Variance     34.67
PSD_8_dyn               29.34
PSD_8_Variance          16.12
ACF_1_8_dyn             22.65
ACF_1_8_Variance       245.05
WL_C1_8_dyn             23.22
WL_C1_8_Variance        12.18
WL_C2_8_dyn              8.53
WL_C2_8_Variance        15.98
WL_C3_8_dyn              6.31
WL_C3_8_Variance        18.89
dtype: float64

##### CatBoost

In [26]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
model = 'CatBoost'
target = 'Flag'
horizons = list(range(1, 16))
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 50))

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model)

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model)
res_means

100%|██████████| 15/15 [18:29:39<00:00, 4438.65s/it]  


Train size  Test size  Train AUC  Test AUC  \
Horizon 1 Share 1 Share real                                               
1       0.05    0.050160          8740.0     2185.0   0.962275  0.954885   
        0.10    0.100293          4371.0     1093.0   0.961345  0.955195   
        0.15    0.150425          2914.0      729.0   0.960635  0.956435   
        0.20    0.200512          2186.0      547.0   0.960280  0.954660   
2       0.05    0.050328         17421.0     4356.0   0.943730  0.923665   
        0.10    0.100615          8714.0     2179.0   0.937800  0.921060   
        0.15    0.150881          5811.0     1453.0   0.940600  0.923815   
        0.20    0.201101          4360.0     1090.0   0.941345  0.921960   
3       0.05    0.050495         26046.0     6512.0   0.938665  0.902265   
        0.10    0.100933         13030.0     3258.0   0.943270  0.904335   
        0.15    0.151325          8691.0     2173.0   0.948685  0.904850   
        0.20    0.201644          6522.0     1631.0   0.946945  0.904640   
4       0.05    0.050661         34614.0     8654.0   0.922430  0.882800   
        0.10    0.101252         17319.0     4330.0   0.931560  0.885170   
        0.15    0.151769         11554.0     2889.0   0.933995  0.886230   
        0.20    0.202214          8672.0     2168.0   0.936725  0.885610   
5       0.05    0.050829         43124.0    10782.0   0.905015  0.863765   
        0.10    0.101568         21581.0     5396.0   0.913435  0.866485   
        0.15    0.152222         14400.0     3600.0   0.915735  0.865990   
        0.20    0.202783         10809.0     2703.0   0.918270  0.866910   
6       0.05    0.050999         51577.0    12895.0   0.892850  0.852995   
        0.10    0.101890         25816.0     6454.0   0.899670  0.855580   
        0.15    0.152675         17228.0     4308.0   0.902445  0.855250   
        0.20    0.203352         12935.0     3234.0   0.904370  0.854635   
7       0.05    0.051169         59973.0    14994.0   0.881650  0.842190   
        0.10    0.102212         30024.0     7506.0   0.888020  0.845230   
        0.15    0.153128         20040.0     5011.0   0.890685  0.845220   
        0.20    0.203923         15048.0     3763.0   0.891860  0.845005   
8       0.05    0.051340         68312.0    17079.0   0.872890  0.836765   
        0.10    0.102535         34204.0     8552.0   0.878550  0.838620   
        0.15    0.153587         22835.0     5709.0   0.881445  0.839725   
        0.20    0.204497         17150.0     4288.0   0.882085  0.836995   
9       0.05    0.051513         76593.0    19149.0   0.865720  0.830160   
        0.10    0.102861         38358.0     9590.0   0.870095  0.830605   
        0.15    0.154048         25612.0     6404.0   0.871980  0.831950   
        0.20    0.205073         19240.0     4810.0   0.872775  0.830860   
10      0.05    0.051687         84817.0    21205.0   0.859985  0.826515   
        0.10    0.103190         42484.0    10622.0   0.863435  0.827635   
        0.15    0.154510         28373.0     7094.0   0.865855  0.826210   
        0.20    0.205652         21317.0     5330.0   0.866415  0.825055   
11      0.05    0.051862         92984.0    23247.0   0.853530  0.822125   
        0.10    0.103521         46584.0    11646.0   0.857755  0.823820   
        0.15    0.154977         31116.0     7780.0   0.858765  0.823155   
        0.20    0.206234         23383.0     5846.0   0.860445  0.820410   
12      0.05    0.052038        101094.0    25274.0   0.847470  0.815985   
        0.10    0.103853         50656.0    12664.0   0.852350  0.819380   
        0.15    0.155446         33843.0     8461.0   0.854280  0.819015   
        0.20    0.206818         25436.0     6360.0   0.854090  0.816905   
13      0.05    0.052216        109147.0    27287.0   0.843065  0.813915   
        0.10    0.104189         54700.0    13676.0   0.847300  0.815135   
        0.15    0.155920         36552.0     9138.0   0.848415  0.814365   
  

In [27]:
# Count an average feature importance
res = pd.read_parquet(f'Params/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 16))]
groups = ['Horizon']

# Get the final pivot
means = round(res_viable[coeffs_cols].mean(), 2)
print(round(means.mean(), 2), round(means.median(), 2))
means

3.33 2.65


Hurst                   0.55
CorrDim                 1.15
Lyapunov                1.87
Skewness                3.92
Kurtosis                3.09
PSD                     6.00
ACF_1                   3.08
WL_C1                   2.24
WL_C2                   2.49
WL_C3                   1.61
Hurst_8_dyn             0.33
Hurst_8_Variance        1.08
CorrDim_8_dyn           0.35
CorrDim_8_Variance      3.11
Lyapunov_8_dyn          0.49
Lyapunov_8_Variance     2.81
Skewness_8_dyn         10.89
Skewness_8_Variance     7.92
Kurtosis_8_dyn          7.93
Kurtosis_8_Variance     5.02
PSD_8_dyn               7.25
PSD_8_Variance          2.40
ACF_1_8_dyn             2.28
ACF_1_8_Variance        9.46
WL_C1_8_dyn             1.82
WL_C1_8_Variance        2.82
WL_C2_8_dyn             0.58
WL_C2_8_Variance        2.97
WL_C3_8_dyn             0.55
WL_C3_8_Variance        3.92
dtype: float64

#### Gradient boosting with Optuna and SHAP

In [27]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose horizon and set random state
horizons = list(range(1, 2))
state = 2025

# Iterate over the chosen parameters and optimize gradient boosting classification models
fun.optuna_and_boosting(data_logdyn, horizons, state)


 LightGBM, 1 horizon:


  0%|          | 0/5 [00:00<?, ?it/s]


 XGBoost, 1 horizon:


  0%|          | 0/5 [00:00<?, ?it/s]


 CatBoost, 1 horizon:


  0%|          | 0/5 [00:00<?, ?it/s]

### Solar data

#### Data and params

In [8]:
# Fix params for the boostrap
target = 'Flag'
horizons = list(range(4, 9))
shares = np.linspace(0.05, 0.07, 3)
states = list(range(0, 10000, 100))
# horizons = [4]
# shares = [0.05]
# states = list(range(0, 10000, 1000))
# states = [0]
path = 'Params_solar'

# Read dataset
data_logdyn = pd.read_parquet('Data/solar_metrics_smoothed.parquet')

##### Probit

In [9]:
# Choose binary target and other parameters
model = 'Probit'

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model)

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols) + ['const'],
                             model = model,
                             path = path)
res_means

100%|██████████| 5/5 [49:34<00:00, 594.94s/it]


Train size  Validation size  Test size  \
Horizon 1 Share 1 Share real                                           
4       0.05    0.050679          8856.0           1688.0     2637.0   
        0.06    0.060805          7381.0           1407.0     2198.0   
        0.07    0.070928          6328.0           1206.0     1884.0   
5       0.05    0.050853         11034.0           2102.0     3284.0   
        0.06    0.061011          9196.0           1752.0     2738.0   
        0.07    0.071167          7884.0           1502.0     2347.0   
6       0.05    0.051026         13195.0           2514.0     3928.0   
        0.06    0.061217         10998.0           2096.0     3274.0   
        0.07    0.071403          9429.0           1797.0     2807.0   
7       0.05    0.051202         15341.0           2923.0     4567.0   
        0.06    0.061426         12788.0           2436.0     3807.0   
        0.07    0.071647         10963.0           2089.0     3264.0   
8       0.05    0.051379         17473.0           3329.0     5201.0   
        0.06    0.061638         14565.0           2775.0     4335.0   
        0.07    0.071890         12488.0           2379.0     3717.0   

                              Train AUC  Validation AUC  Test AUC  \
Horizon 1 Share 1 Share real                                        
4       0.05    0.050679        0.92077         0.91462   0.91675   
        0.06    0.060805        0.92188         0.91810   0.91399   
        0.07    0.070928        0.92312         0.91541   0.91498   
5       0.05    0.050853        0.90661         0.90142   0.90283   
        0.06    0.061011        0.90874         0.90268   0.90023   
        0.07    0.071167        0.90953         0.90286   0.90105   
6       0.05    0.051026        0.89092         0.88451   0.88884   
        0.06    0.061217        0.89148         0.88284   0.88742   
        0.07    0.071403        0.89236         0.88803   0.88877   
7       0.05    0.051202        0.87481         0.86907   0.86612   
        0.06    0.061426        0.87449         0.87033   0.87143   
        0.07    0.071647        0.87476         0.87274   0.87240   
8       0.05    0.051379        0.85634         0.85152   0.85071   
        0.06    0.061638        0.85742         0.85012   0.85267   
        0.07    0.071890        0.85778         0.85099   0.85142   

                              Train KS-test p-value  \
Horizon 1 Share 1 Share real                          
4       0.05    0.050679                        0.0   
        0.06    0.060805                        0.0   
        0.07    0.070928                        0.0   
5       0.05    0.050853                        0.0   
        0.06    0.061011                        0.0   
        0.07    0.071167                        0.0   
6       0.05    0.051026                        0.0   
        0.06    0.061217                        0.0   
        0.07    0.071403                        0.0   
7       0.05    0.051202                        0.0   
        0.06    0.061426                        0.0   
        0.07    0.071647                        0.0   
8       0.05    0.051379                        0.0   
        0.06    0.061638                        0.0   
        0.07    0.071890                        0.0   

                              Validation KS-test p-value  \
Horizon 1 Share 1 Share real                               
4       0.05    0.050679                             0.0   
        0.06    0.060805                             0.0   
        0.07    0.070928                             0.0   
5       0.05    0.050853                             0.0   
        0.06    0.061011                             0.0   
        0.07    0.071167                             0.0   
6       0.05    0.051026                             0.0   
        0.06    0.061217                             0.0   
        0.07    0.071403                             0.0   
7       0.05    0.051202           

In [10]:
# Count a share of appearances for positive horizons
res = pd.read_parquet(f'{path}/params_Probit.parquet')
groups = ['Horizon', '1 Share', '1 Share real']
drops = ['State']
round(res.groupby(groups)[list(coeffs_cols) + ['const']].count() / 20, 2)

Hurst  CorrDim  Lyapunov  Skewness  Kurtosis  \
Horizon 1 Share 1 Share real                                                 
4       0.05    0.050679       0.40     0.55      0.25       5.0      4.40   
        0.06    0.060805       0.15     0.30      0.30       5.0      3.75   
        0.07    0.070928       0.10     0.50      0.15       5.0      3.65   
5       0.05    0.050853       0.30     1.00      0.75       5.0      4.70   
        0.06    0.061011       0.30     0.70      0.40       5.0      4.50   
        0.07    0.071167       0.45     0.85      0.45       5.0      3.85   
6       0.05    0.051026       0.45     2.45      0.25       5.0      4.80   
        0.06    0.061217       0.40     2.75      0.30       5.0      5.00   
        0.07    0.071403       0.40     1.95      0.20       5.0      4.50   
7       0.05    0.051202       0.60     3.85      0.50       5.0      5.00   
        0.06    0.061426       0.80     3.05      0.45       5.0      4.95   
        0.07    0.071647       0.75     3.35      0.55       5.0      4.90   
8       0.05    0.051379       1.60     4.05      1.55       5.0      4.95   
        0.06    0.061638       2.10     4.00      1.00       5.0      4.95   
        0.07    0.071890       1.35     4.35      0.90       5.0      4.90   

                              PSD  ACF_1  WL_C1  WL_C2  WL_C3  Hurst_8_dyn  \
Horizon 1 Share 1 Share real                                                 
4       0.05    0.050679      5.0    5.0   0.00   1.05   0.15         0.15   
        0.06    0.060805      5.0    5.0   0.00   0.80   0.25         0.15   
        0.07    0.070928      5.0    5.0   0.05   0.25   0.00         0.10   
5       0.05    0.050853      5.0    5.0   0.05   1.25   0.00         0.15   
        0.06    0.061011      5.0    5.0   0.00   0.95   0.05         0.00   
        0.07    0.071167      5.0    5.0   0.00   0.45   0.00         0.05   
6       0.05    0.051026      5.0    5.0   0.05   1.90   0.15         0.05   
        0.06    0.061217      5.0    5.0   0.00   1.30   0.05         0.05   
        0.07    0.071403      5.0    5.0   0.05   1.00   0.05         0.00   
7       0.05    0.051202      5.0    5.0   0.00   2.35   0.05         0.00   
        0.06    0.061426      5.0    5.0   0.05   2.05   0.10         0.00   
        0.07    0.071647      5.0    5.0   0.00   1.15   0.05         0.10   
8       0.05    0.051379      5.0    5.0   0.50   1.65   0.00         0.15   
        0.06    0.061638      5.0    5.0   0.60   1.40   0.20         0.10   
        0.07    0.071890      5.0    5.0   0.55   1.20   0.15         0.10   

                              Hurst_8_Variance  CorrDim_8_dyn  \
Horizon 1 Share 1 Share real                                    
4       0.05    0.050679                  0.00           1.10   
        0.06    0.060805                  0.10           1.65   
        0.07    0.070928                  0.15           1.40   
5       0.05    0.050853                  0.00           1.50   
        0.06    0.061011                  0.05           1.00   
        0.07    0.071167                  0.00           1.05   
6       0.05    0.051026                  0.05           0.90   
        0.06    0.061217                  0.00           0.75   
        0.07    0.071403                  0.15           0.70   
7       0.05    0.051202                  0.00           0.50   
        0.06    0.061426                  0.10           0.65   
        0.07    0.071647                  0.00           0.55   
8       0.05    0.051379                  0.00           1.00   
        0.06    0.061638                  0.00           0.50   
        0.07    0.071890                  0.00           0.65   

                              CorrDim_8_Variance  Lyapunov_8_dyn  \
Horizon 1 Share 1 Share real                                       
4       0.05    0.050679                    3.35             0.0   
        0.06    0.060805                    3.35             0.0  

In [11]:
# Count an aggregated share of appearances
pd.DataFrame(round(res[list(coeffs_cols) + ['const']].count() / len(res), 2), columns = ['Share'])

,Share
Hurst,0.14
CorrDim,0.45
Lyapunov,0.11
Skewness,1.00
Kurtosis,0.92
PSD,1.00
ACF_1,1.00
WL_C1,0.03
WL_C2,0.25
WL_C3,0.02


##### LightGBM

In [12]:
# Choose binary target and other parameters
model = 'LightGBM'

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model, target_metric = 'Precision')

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model,
                             path = path)
res_means

100%|██████████| 5/5 [16:18<00:00, 195.77s/it]


Train size  Validation size  Test size  \
Horizon 1 Share 1 Share real                                           
4       0.05    0.050679          8856.0           1688.0     2637.0   
        0.06    0.060805          7381.0           1407.0     2198.0   
        0.07    0.070928          6328.0           1206.0     1884.0   
5       0.05    0.050853         11034.0           2102.0     3284.0   
        0.06    0.061011          9196.0           1752.0     2738.0   
        0.07    0.071167          7884.0           1502.0     2347.0   
6       0.05    0.051026         13195.0           2514.0     3928.0   
        0.06    0.061217         10998.0           2096.0     3274.0   
        0.07    0.071403          9429.0           1797.0     2807.0   
7       0.05    0.051202         15341.0           2923.0     4567.0   
        0.06    0.061426         12788.0           2436.0     3807.0   
        0.07    0.071647         10963.0           2089.0     3264.0   
8       0.05    0.051379         17473.0           3329.0     5201.0   
        0.06    0.061638         14565.0           2775.0     4335.0   
        0.07    0.071890         12488.0           2379.0     3717.0   

                              Train AUC  Validation AUC  Test AUC  \
Horizon 1 Share 1 Share real                                        
4       0.05    0.050679        0.98640         0.97037   0.96980   
        0.06    0.060805        0.98651         0.97153   0.96979   
        0.07    0.070928        0.98686         0.96992   0.97002   
5       0.05    0.050853        0.98388         0.96737   0.96654   
        0.06    0.061011        0.98354         0.96814   0.96641   
        0.07    0.071167        0.98432         0.96836   0.96728   
6       0.05    0.051026        0.97755         0.96100   0.96234   
        0.06    0.061217        0.97771         0.96038   0.96246   
        0.07    0.071403        0.97841         0.96142   0.96161   
7       0.05    0.051202        0.96996         0.95251   0.95151   
        0.06    0.061426        0.97030         0.95352   0.95368   
        0.07    0.071647        0.97043         0.95335   0.95396   
8       0.05    0.051379        0.96271         0.94592   0.94294   
        0.06    0.061638        0.96291         0.94381   0.94359   
        0.07    0.071890        0.96281         0.94373   0.94403   

                              Train KS-test p-value  \
Horizon 1 Share 1 Share real                          
4       0.05    0.050679                        0.0   
        0.06    0.060805                        0.0   
        0.07    0.070928                        0.0   
5       0.05    0.050853                        0.0   
        0.06    0.061011                        0.0   
        0.07    0.071167                        0.0   
6       0.05    0.051026                        0.0   
        0.06    0.061217                        0.0   
        0.07    0.071403                        0.0   
7       0.05    0.051202                        0.0   
        0.06    0.061426                        0.0   
        0.07    0.071647                        0.0   
8       0.05    0.051379                        0.0   
        0.06    0.061638                        0.0   
        0.07    0.071890                        0.0   

                              Validation KS-test p-value  \
Horizon 1 Share 1 Share real                               
4       0.05    0.050679                             0.0   
        0.06    0.060805                             0.0   
        0.07    0.070928                             0.0   
5       0.05    0.050853                             0.0   
        0.06    0.061011                             0.0   
        0.07    0.071167                             0.0   
6       0.05    0.051026                             0.0   
        0.06    0.061217                             0.0   
        0.07    0.071403                             0.0   
7       0.05    0.051202           

In [13]:
# Count an average feature importance
res = pd.read_parquet(f'{path}/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 16))]
groups = ['Horizon']

# Get the final pivot
means = round(res_viable[coeffs_cols].mean(), 2)
print(f'Mean feature importance: {round(means.mean(), 2)}, Median feature importance: {round(means.median(), 2)}')
means

Mean feature importance: 563.13, Median feature importance: 78.47


Hurst                    191.73
CorrDim                  113.25
Lyapunov                  88.74
Skewness                 636.15
Kurtosis                 590.66
PSD                      435.97
ACF_1                   1548.49
WL_C1                     74.40
WL_C2                     27.09
WL_C3                     32.49
Hurst_8_dyn               35.07
Hurst_8_Variance          60.42
CorrDim_8_dyn             47.08
CorrDim_8_Variance        68.80
Lyapunov_8_dyn            18.60
Lyapunov_8_Variance       94.64
Skewness_8_dyn            40.72
Skewness_8_Variance       82.54
Kurtosis_8_dyn            72.64
Kurtosis_8_Variance      114.30
PSD_8_dyn                151.35
PSD_8_Variance           106.13
ACF_1_8_dyn            10504.70
ACF_1_8_Variance        1505.82
WL_C1_8_dyn               21.24
WL_C1_8_Variance          37.74
WL_C2_8_dyn               15.41
WL_C2_8_Variance          23.99
WL_C3_8_dyn                7.33
WL_C3_8_Variance         146.34
dtype: float64

##### XGBoost

In [14]:
# Choose binary target and other parameters
model = 'XGBoost'

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states, model = model, target_metric = 'Precision')

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model,
                             path = path)
res_means

100%|██████████| 5/5 [07:12<00:00, 86.43s/it]


Train size  Validation size  Test size  \
Horizon 1 Share 1 Share real                                           
4       0.05    0.050679          8856.0           1688.0     2637.0   
        0.06    0.060805          7381.0           1407.0     2198.0   
        0.07    0.070928          6328.0           1206.0     1884.0   
5       0.05    0.050853         11034.0           2102.0     3284.0   
        0.06    0.061011          9196.0           1752.0     2738.0   
        0.07    0.071167          7884.0           1502.0     2347.0   
6       0.05    0.051026         13195.0           2514.0     3928.0   
        0.06    0.061217         10998.0           2096.0     3274.0   
        0.07    0.071403          9429.0           1797.0     2807.0   
7       0.05    0.051202         15341.0           2923.0     4567.0   
        0.06    0.061426         12788.0           2436.0     3807.0   
        0.07    0.071647         10963.0           2089.0     3264.0   
8       0.05    0.051379         17473.0           3329.0     5201.0   
        0.06    0.061638         14565.0           2775.0     4335.0   
        0.07    0.071890         12488.0           2379.0     3717.0   

                              Train AUC  Validation AUC  Test AUC  \
Horizon 1 Share 1 Share real                                        
4       0.05    0.050679        0.97251         0.96023   0.95990   
        0.06    0.060805        0.96869         0.95957   0.95504   
        0.07    0.070928        0.96555         0.95361   0.95181   
5       0.05    0.050853        0.97121         0.95898   0.95785   
        0.06    0.061011        0.96640         0.95525   0.95274   
        0.07    0.071167        0.96387         0.95144   0.95008   
6       0.05    0.051026        0.96489         0.95037   0.95132   
        0.06    0.061217        0.96217         0.94888   0.94949   
        0.07    0.071403        0.95394         0.94019   0.94133   
7       0.05    0.051202        0.95053         0.93578   0.93442   
        0.06    0.061426        0.94748         0.93517   0.93559   
        0.07    0.071647        0.95196         0.93909   0.93999   
8       0.05    0.051379        0.95476         0.93966   0.93699   
        0.06    0.061638        0.95024         0.93428   0.93348   
        0.07    0.071890        0.94417         0.92885   0.92946   

                              Train KS-test p-value  \
Horizon 1 Share 1 Share real                          
4       0.05    0.050679                        0.0   
        0.06    0.060805                        0.0   
        0.07    0.070928                        0.0   
5       0.05    0.050853                        0.0   
        0.06    0.061011                        0.0   
        0.07    0.071167                        0.0   
6       0.05    0.051026                        0.0   
        0.06    0.061217                        0.0   
        0.07    0.071403                        0.0   
7       0.05    0.051202                        0.0   
        0.06    0.061426                        0.0   
        0.07    0.071647                        0.0   
8       0.05    0.051379                        0.0   
        0.06    0.061638                        0.0   
        0.07    0.071890                        0.0   

                              Validation KS-test p-value  \
Horizon 1 Share 1 Share real                               
4       0.05    0.050679                             0.0   
        0.06    0.060805                             0.0   
        0.07    0.070928                             0.0   
5       0.05    0.050853                             0.0   
        0.06    0.061011                             0.0   
        0.07    0.071167                             0.0   
6       0.05    0.051026                             0.0   
        0.06    0.061217                             0.0   
        0.07    0.071403                             0.0   
7       0.05    0.051202           

In [15]:
# Count an average feature importance
res = pd.read_parquet(f'{path}/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 16))]
groups = ['Horizon']

# Get the final pivot
means = round(res_viable[coeffs_cols].mean(), 2)
print(f'Mean feature importance: {round(means.mean(), 2)}, Median feature importance: {round(means.median(), 2)}')
means

Mean feature importance: 15.86, Median feature importance: 7.96


Hurst                    9.56
CorrDim                  8.19
Lyapunov                 7.48
Skewness                12.97
Kurtosis                16.15
PSD                     14.51
ACF_1                   31.66
WL_C1                    9.09
WL_C2                    7.90
WL_C3                    5.65
Hurst_8_dyn              6.42
Hurst_8_Variance         6.58
CorrDim_8_dyn            8.30
CorrDim_8_Variance       7.98
Lyapunov_8_dyn           5.69
Lyapunov_8_Variance      7.20
Skewness_8_dyn           8.47
Skewness_8_Variance      7.01
Kurtosis_8_dyn           7.95
Kurtosis_8_Variance     11.47
PSD_8_dyn                8.39
PSD_8_Variance           7.68
ACF_1_8_dyn            193.43
ACF_1_8_Variance        26.22
WL_C1_8_dyn              6.78
WL_C1_8_Variance         6.60
WL_C2_8_dyn              6.86
WL_C2_8_Variance         6.06
WL_C3_8_dyn              5.43
WL_C3_8_Variance         8.01
dtype: float64

##### CatBoost

In [16]:
# Choose binary target and other parameters
model = 'CatBoost'

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
res = fun.model(data_logdyn, target, horizons, shares, states = list(range(0, 10000, 500)), model = model, target_metric = 'Precision')

# Save results and create pivot based on the horizon and 1 share parameters
coeffs_cols = data_logdyn.columns.drop(['Volume', 'MA100', 'MV100', 'Rise', 'Distance', 'Index', 'Ticker'])
res_means = fun.save_results(res, 
                             list(coeffs_cols),
                             model = model,
                             path = path)
res_means

100%|██████████| 5/5 [04:00<00:00, 48.15s/it]


Train size  Validation size  Test size  \
Horizon 1 Share 1 Share real                                           
4       0.05    0.050679          8856.0           1688.0     2637.0   
        0.06    0.060805          7381.0           1407.0     2198.0   
        0.07    0.070928          6328.0           1206.0     1884.0   
5       0.05    0.050853         11034.0           2102.0     3284.0   
        0.06    0.061011          9196.0           1752.0     2738.0   
        0.07    0.071167          7884.0           1502.0     2347.0   
6       0.05    0.051026         13195.0           2514.0     3928.0   
        0.06    0.061217         10998.0           2096.0     3274.0   
        0.07    0.071403          9429.0           1797.0     2807.0   
7       0.05    0.051202         15341.0           2923.0     4567.0   
        0.06    0.061426         12788.0           2436.0     3807.0   
        0.07    0.071647         10963.0           2089.0     3264.0   
8       0.05    0.051379         17473.0           3329.0     5201.0   
        0.06    0.061638         14565.0           2775.0     4335.0   
        0.07    0.071890         12488.0           2379.0     3717.0   

                              Train AUC  Validation AUC  Test AUC  \
Horizon 1 Share 1 Share real                                        
4       0.05    0.050679        0.92300         0.90775   0.91105   
        0.06    0.060805        0.93610         0.93455   0.93340   
        0.07    0.070928        0.90205         0.89740   0.89225   
5       0.05    0.050853        0.91010         0.90515   0.90505   
        0.06    0.061011        0.94225         0.93605   0.93740   
        0.07    0.071167        0.91180         0.89935   0.90515   
6       0.05    0.051026        0.92920         0.92345   0.92290   
        0.06    0.061217        0.93015         0.92150   0.92380   
        0.07    0.071403        0.94030         0.93200   0.93700   
7       0.05    0.051202        0.92950         0.91970   0.91780   
        0.06    0.061426        0.89990         0.89265   0.89335   
        0.07    0.071647        0.90935         0.90160   0.90920   
8       0.05    0.051379        0.92295         0.91710   0.91965   
        0.06    0.061638        0.91605         0.90445   0.90715   
        0.07    0.071890        0.92825         0.92365   0.91880   

                              Train KS-test p-value  \
Horizon 1 Share 1 Share real                          
4       0.05    0.050679                        0.0   
        0.06    0.060805                        0.0   
        0.07    0.070928                        0.0   
5       0.05    0.050853                        0.0   
        0.06    0.061011                        0.0   
        0.07    0.071167                        0.0   
6       0.05    0.051026                        0.0   
        0.06    0.061217                        0.0   
        0.07    0.071403                        0.0   
7       0.05    0.051202                        0.0   
        0.06    0.061426                        0.0   
        0.07    0.071647                        0.0   
8       0.05    0.051379                        0.0   
        0.06    0.061638                        0.0   
        0.07    0.071890                        0.0   

                              Validation KS-test p-value  \
Horizon 1 Share 1 Share real                               
4       0.05    0.050679                             0.0   
        0.06    0.060805                             0.0   
        0.07    0.070928                             0.0   
5       0.05    0.050853                             0.0   
        0.06    0.061011                             0.0   
        0.07    0.071167                             0.0   
6       0.05    0.051026                             0.0   
        0.06    0.061217                             0.0   
        0.07    0.071403                             0.0   
7       0.05    0.051202           

In [17]:
# Count an average feature importance
res = pd.read_parquet(f'{path}/params_{model}.parquet')
res_viable = res[res['Horizon'].isin(range(1, 16))]
groups = ['Horizon']

# Get the final pivot
means = round(res_viable[coeffs_cols].mean(), 2)
print(f'Mean feature importance: {round(means.mean(), 2)}, Median feature importance: {round(means.median(), 2)}')
means

Mean feature importance: 3.33, Median feature importance: 0.5


Hurst                   1.62
CorrDim                 0.75
Lyapunov                0.43
Skewness                2.61
Kurtosis                3.09
PSD                     2.83
ACF_1                  22.22
WL_C1                   0.41
WL_C2                   0.18
WL_C3                   0.15
Hurst_8_dyn             0.13
Hurst_8_Variance        0.13
CorrDim_8_dyn           0.34
CorrDim_8_Variance      0.49
Lyapunov_8_dyn          0.20
Lyapunov_8_Variance     0.53
Skewness_8_dyn          0.52
Skewness_8_Variance     2.72
Kurtosis_8_dyn          0.47
Kurtosis_8_Variance     1.00
PSD_8_dyn               2.07
PSD_8_Variance          0.65
ACF_1_8_dyn            38.72
ACF_1_8_Variance       16.42
WL_C1_8_dyn             0.19
WL_C1_8_Variance        0.17
WL_C2_8_dyn             0.06
WL_C2_8_Variance        0.13
WL_C3_8_dyn             0.05
WL_C3_8_Variance        0.68
dtype: float64